## Plan an Outing

In [ ]:
import ipyleaflet as ipl
import ipywidgets as widgets
import toml
import json
import requests
import collections
import arrow

from IPython.core.display import HTML

# See readme.md for credentials.toml formatting
cred = toml.load('credentials.toml')
appid = cred['api']['here']['appid']
appcode = cred['api']['here']['appcode']
walk_10_min = cred['minute']['ten']['polygon']

polygon = []
for item in walk_10_min:
    item = tuple(item)
    polygon.append(item)

def open_file(library_only=True):
    
    try:
        with open('library.geo.json', 'r') as l:
            collection = json.load(l)

        if library_only:
            library = collection['features']
            return library
        else:
            return collection

    except:
        return None

library = open_file(library_only=True)
collection = open_file(library_only=False)

def address_to_coord(address_string):
    """Return a tuple of the Lat/Long of the address."""
    raw_address = '+'.join(address_string.replace(',', ' ').split(' '))
    formatted_address = raw_address.replace('++', '+')

    url = "https://geocoder.api.here.com/6.2/geocode.json" +\
        "?app_id=" +\
        appid +\
        "&app_code=" +\
        appcode +\
        "&searchtext=" +\
        formatted_address

    response = requests.get(url)

    data = response.json()
    lat_long = data['Response']['View'][0]['Result'][0]['Location']['NavigationPosition'][0]
    point = coord_to_tuple(lat_long)
        
    return point

def plot_isochrones(coordinates, isochrones, zoom_level=8):
    '''Initiate map and plot isochrones centered on coordinates.'''

    center = coord_to_tuple(coordinates)

    layer_color = 'brown'
    fill_color = 'orange'
    m = ipl.Map(center=center, zoom=zoom_level)
    for key in isochrones.keys():

        item = ipl.Polygon(
            locations=isochrones[key],
            color=layer_color,
            weight=1,
            fill_color=fill_color,
            fill_opacity=0.2,
            name=key)
        m.add_layer(item)

    m.add_control(ipl.LayersControl())

    return m

def coord_to_tuple(coordinates):
    """Transform coordinates to a 2 tuple."""
    if type(coordinates) == str:
        center = tuple(map(float, coordinates.split(',')))

    elif type(coordinates) == dict:
        center = tuple(coordinates.values())

    elif type(coordinates) == tuple:
        center = coordinates

    elif (type(coordinates) == list) and (len(coordinates) == 2):
        center = tuple(coordinates)

    else:
        print('invalid coordinate type')
        center = None

    return center

def coord_to_string(coordinates):
    """Transform coordinates into a string for API"""
    if type(coordinates) == tuple:
        string = ','.join(list(map(str, coordinates)))
        
    elif type(coordinates) == dict:
        string = ','.join(list(map(str, coordinates.values())))
    
    elif (type(coordinates) == list) and (len(coordinates) == 2):
        string = ','.join(list(map(str, coordinates)))
        
    elif type(coordinates) == str:
        string = coordinates
        
    else:
        print('invalid coordinate type')
        string = None        

    return string

def item_to_popup(new_item):
    
    output_string = str()
    
    for key,value in new_item['properties'].items():

        output_string = output_string + "<b>" + key.title() + ":</b>  " + str(value) + "</br>"
    
    return output_string


def display_results(x):

    
    out.clear_output(wait=True)
    #map_result = create_map(address=address_box.value,
      #                      mode=method.value)
    
    center = address_to_coord(address_box.value)
    
    global m
    m = ipl.Map(center=center, zoom=16)

    global marker
    #marker= ipl.Marker(location=center, draggable=True)
    #m.add_layer(marker)
    
    poly = ipl.Polygon(
                locations=polygon,
                color='brown',
                weight=1,
                fill=False,
                name='10 Minute Walking Distance',)
    m.add_layer(poly)

    
    for item in collection['features']:

        location = item['geometry']['coordinates']

        item_properties = widgets.HTML()



        item_popup = item_to_popup(item)

        item_properties.value = item_popup

        item_marker = ipl.Marker(location=location)

        item_marker.title = item['properties']['name']
        item_marker.name = item['properties']['name']
        item_marker.popup = item_properties

        marker_layer_name = item['properties']['name'].replace(" ","_")
        

        vars()[marker_layer_name] = item_marker


        m.add_layer(vars()[marker_layer_name])
    
    m.add_control(ipl.LayersControl())
    
    with out:
        display(m,trip_box)
        

submit = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Submit',
    icon=''
)


trip_name_box = widgets.Text(
    value='',
    placeholder='Name the Trip',
    description='Trip Name:',
    layout=widgets.Layout(width='auto'),
    disabled=False
)


address_box = widgets.Text(
    value='213 Main Street, Graham, NC, 27253',
    placeholder='213 Main Street, Graham, NC, 27253',
    description='Address:',
    layout=widgets.Layout(width='auto'),
    disabled=False
)

art_type = widgets.SelectMultiple(
    options=sorted(collection['metadata']['art_types']),
    value=['Sculpture'],
    rows=6,
    description='Type of Art:',
    disabled=False
)

trip_description_box = widgets.Textarea(
    value='',
    placeholder='Describe your trip. For instance "Murals and sculptures in downtown Graham."',
    layout=widgets.Layout(width='auto'),
    description='Trip Description:',
    disabled=False
)



trip_warnings_select = widgets.SelectMultiple(
    options=sorted(collection['metadata']['warnings']),
    value=['None'],
    rows=6,
    description='Exclude:',
    disabled=False
)

save = widgets.Button(
    description='Save',
    disabled=False,
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Submit',
    icon=''
)

create_pdf = widgets.Button(
    description='Output to PDF',
    disabled=False,
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Submit',
    icon=''
)


initial = [address_box, art_type, trip_warnings_select, submit]

trip_item = [trip_name_box, address_box, trip_description_box, save, create_pdf]

box_layout = widgets.Layout(display='flex',
                            flex_flow='column',
                            align_items='stretch',
                            border='dash',
                            width='70%')

box = widgets.Box(children=initial, layout=box_layout)
trip_box = widgets.Box(children=trip_item, layout=box_layout)


out = widgets.Output()

with out:
    display(box)
    


submit.on_click(display_results)    

#save.on_click(save_new_item)

out